# 03. Model v0

Model v0 is an ML model for typical V-NLI, which accepts text and tabular data as input, and returns the corresponding figure.

We adopt [TAPEX](https://arxiv.org/abs/2107.07653), a pre-trained [BART](https://arxiv.org/abs/1910.13461) model, as a base model.
We use almost the same hyperparameters as TAPEX, and fine-tune the model with the nvBench dataset preprocessed in the previous notebook.
For implementation, we use Hugging Face Transformers, one of the most famous NLP libraries.
Primarily we refer to [this TAPEX example](https://github.com/huggingface/transformers/blob/main/examples/research_projects/tapex/run_wikisql_with_tapex.py
).

As a result, the exact match ratio gets to be ~60% in the test dataset.
You can check the experiment results on [Weights & Biases](https://wandb.ai/kwkty/vxnli/runs/28s2s1t8/overview).
It's not comparable to the existing work because we adopt a different way to preprocess the nvBench dataset.
However, it doesn't matter so much because our goal is not to improve an ML model for V-NLI but to propose a novel UI for data visualization.

In the final user study, we use this model as the baseline model to compare our proposed interface with typical V-NLI.

## Setup

### Define Parameters


In [1]:
data_dir: str = "../data/"
push_model_to_huggingface_hub: bool = True
use_wandb: bool = True


### Load Modules

In [2]:
import functools
import multiprocessing
import os
import pandas as pd
import sqlite3

from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List

import evaluate
import datasets
import numpy as np
import torch
import transformers

from datasets import Dataset
from transformers import (
    BartConfig,
    BartForConditionalGeneration,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
    EvalPrediction,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TapexTokenizer,
    trainer_utils,
)


In [3]:
transformers.set_seed(123)


### Define Variables

In [4]:
# Paths

DATA_DIR: Path = Path(data_dir)

DATABASE_DIR: Path = DATA_DIR.joinpath("datasets/nvBench/database")
DATASET_DIR: Path = DATA_DIR.joinpath("datasets/datasets/vxnli-v0/")
DATASET_OUTPUT_DIR: Path = DATA_DIR.joinpath(f"datasets/vxnli-v0.hf/")

MODEL_NAME: str = "vxnli-v0"
MODEL_OUTPUT_DIR: Path = DATA_DIR.joinpath(f"models/{MODEL_NAME}/")
RESULT_OUTPUT_DIR: Path = DATA_DIR.joinpath(f"results/{MODEL_NAME}/")

# Model Parameters

BASE_MODEL: str = "microsoft/tapex-base-finetuned-wtq"
MAX_SOURCE_LENGTH: int = 1024
MAX_TARGET_LENGTH: int = 124


In [5]:
MODEL_OUTPUT_DIR.mkdir(exist_ok=True)
RESULT_OUTPUT_DIR.mkdir(exist_ok=True)


### Load Tokenizer


In [6]:
tokenizer = TapexTokenizer.from_pretrained(
    BASE_MODEL, use_fast=True, add_prefix_space=True
)


In [7]:
# Example
tokenizer(
    table=pd.DataFrame.from_dict(
        {
            "Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"],
            "Number of movies": ["87", "53", "69"],
        }
    ),
    answer="how many movies does Leonardo Di Caprio have?",
    return_tensors="pt",
)


{'input_ids': tensor([[    0, 11311,  4832,  5552,  1721,   346,     9,  4133,  3236,   112,
          4832,  5378,   625,   181,  2582,  1721,  8176,  3236,   132,  4832,
          2084,   261,  6782,  2269,  2927, 12834,  1721,  4268,  3236,   155,
          4832,  5473, 26875, 42771,  6071,  1721,  5913,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

### Load Model

In [8]:
model_config = BartConfig.from_pretrained(
    BASE_MODEL,
    no_repeat_ngram_size=0,
    max_length=MAX_SOURCE_LENGTH,
    early_stopping=False,
)

model = BartForConditionalGeneration.from_pretrained(
    BASE_MODEL,
    config=model_config,
)


## Preprocess Dataset


In [9]:
def load_table(db_id: str, table_name: str) -> pd.DataFrame:
    db_path = DATABASE_DIR.joinpath(f"{db_id}/{db_id}.sqlite")

    with sqlite3.connect(db_path) as con:
        return pd.read_sql(f"SELECT * FROM {table_name}", con)


# Example
load_table("customers_and_products_contacts", "products").head()


,product_id,product_type_code,product_name,product_price
0,1,Hardware,Apple,5.475398e+07
1,2,Clothes,jcrew,3.059093e+07
2,3,Hardware,Apple,1.026885e+04
3,4,Hardware,Apple,2.295667e+07
4,5,Clothes,jcrew,5.927022e+06


In [10]:
def preprocess_table(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={col: col.lower() for col in df.columns})

    # The TAPEX tokenizer raises an error when the table contains non-str columns
    df = df.astype(str)

    for col_name, col_dtype in zip(df.columns, df.dtypes):
        df[col_name] = df[col_name].str.lower()

    return df


preprocess_table(load_table("customers_and_products_contacts", "products").head())


,product_id,product_type_code,product_name,product_price
0,1,hardware,apple,54753982.574522
1,2,clothes,jcrew,30590929.528306
2,3,hardware,apple,10268.85297069
3,4,hardware,apple,22956668.699482
4,5,clothes,jcrew,5927021.8748021


In [11]:
# functools.cache is supported in python3.9+, but use lru_cache to support python3.7+
@functools.lru_cache(maxsize=None)
def load_and_preprocess_table(db_id: str, table_name: str) -> pd.DataFrame:
    table = load_table(db_id, table_name)
    table = preprocess_table(table)

    return table


In [12]:
def preprocess_dataset(example: Dict[str, Any]) -> Dict[str, torch.Tensor]:
    table = load_and_preprocess_table(example["db_id"], example["table"])

    query = example["question"]
    answer = example["vega_zero"]

    model_inputs = tokenizer(
        table=table,
        query=query,
        answer=answer,
        max_length=MAX_SOURCE_LENGTH,
        padding=True,
        truncation=True,
    )

    labels = tokenizer(
        answer=answer,
        max_length=MAX_TARGET_LENGTH,
        padding=True,
        truncation=True,
    )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [13]:
if DATASET_OUTPUT_DIR.exists():
    # load_from_dist doesn't support pathlib.Path
    dataset = datasets.load_from_disk(str(DATASET_OUTPUT_DIR))
else:
    dataset = datasets.load_dataset(
        "csv",
        data_files={
            # load_dataset doesn't support pathlib.Path
            "train": str(DATASET_DIR.joinpath("train.csv")),
            "test": str(DATASET_DIR.joinpath("test.csv")),
            "validation": str(DATASET_DIR.joinpath("val.csv")),
        },
    )

    dataset = dataset.map(
        preprocess_dataset,
        batched=False,
        num_proc=multiprocessing.cpu_count(),
    )

    # save_to_disk doesn't support pathlib.Path
    dataset.save_to_disk(str(DATASET_OUTPUT_DIR))

dataset


DatasetDict({
    train: Dataset({
        features: ['db_id', 'chart', 'hardness', 'query', 'question', 'vega_zero', 'SQL', 'table', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 12798
    })
    test: Dataset({
        features: ['db_id', 'chart', 'hardness', 'query', 'question', 'vega_zero', 'SQL', 'table', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1543
    })
    validation: Dataset({
        features: ['db_id', 'chart', 'hardness', 'query', 'question', 'vega_zero', 'SQL', 'table', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1385
    })
})

## Training


In [14]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=None,
)


In [15]:
exact_match = evaluate.load("exact_match")


def compute_metrics(eval_pred: EvalPrediction):
    preds, labels = eval_pred

    preds = tokenizer.batch_decode(
        preds, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(
        labels, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    return exact_match.compute(predictions=preds, references=labels)


In [16]:
trainer = Seq2SeqTrainer(
    model=model,
    args=Seq2SeqTrainingArguments(
        output_dir=MODEL_OUTPUT_DIR,
        predict_with_generate=True,
        num_train_epochs=50,
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        do_eval=True,
        metric_for_best_model="exact_match",
        push_to_hub=push_model_to_huggingface_hub,
        report_to="wandb" if use_wandb else "none",
    ),
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=5),
    ],
)


/home/jupyter/vxnli/notebooks/../data/models/vxnli-v0 is already a clone of https://huggingface.co/kwkty/vxnli-v0. Make sure you pull the latest changes with `repo.git_pull()`.


In [17]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: query, vega_zero, chart, db_id, SQL, hardness, table, question. If query, vega_zero, chart, db_id, SQL, hardness, table, question are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/home/jupyter/vxnli/.venv/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12798
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 20000
  Number of trainable parameters = 139420416
Automatic

/home/jupyter/vxnli/.venv/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Exact Match
1,0.223800,0.317347,0.549458
2,0.018900,0.345356,0.576895
3,0.011700,0.382002,0.550903
4,0.007700,0.375346,0.576895
5,0.006300,0.391200,0.593502
6,0.005400,0.393180,0.590614
7,0.005200,0.396688,0.574007
8,0.004900,0.389922,0.610108
9,0.004000,0.411082,0.597834
10,0.004500,0.438079,0.587004


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: query, vega_zero, chart, db_id, SQL, hardness, table, question. If query, vega_zero, chart, db_id, SQL, hardness, table, question are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1385
  Batch size = 32
Saving model checkpoint to ../data/models/vxnli-v0/checkpoint-400
Configuration saved in ../data/models/vxnli-v0/checkpoint-400/config.json
Model weights saved in ../data/models/vxnli-v0/checkpoint-400/pytorch_model.bin
tokenizer config file saved in ../data/models/vxnli-v0/checkpoint-400/tokenizer_config.json
Special tokens file saved in ../data/models/vxnli-v0/checkpoint-400/special_tokens_map.json
tokenizer config file saved in ../data/models/vxnli-v0/tokenizer_config.json
Special tokens file saved in ../data/models/vxnli-v0/special_tokens_map.json

TrainOutput(global_step=5200, training_loss=0.023402981368395, metrics={'train_runtime': 3599.3435, 'train_samples_per_second': 177.782, 'train_steps_per_second': 5.557, 'total_flos': 1.0144429350322176e+17, 'train_loss': 0.023402981368395, 'epoch': 13.0})

## Evaluation

In [18]:
# trainer.evaluate must be called for the model card

trainer.evaluate(dataset["test"])


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: query, vega_zero, chart, db_id, SQL, hardness, table, question. If query, vega_zero, chart, db_id, SQL, hardness, table, question are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1543
  Batch size = 32
/home/jupyter/vxnli/.venv/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.34090954065322876,
 'eval_exact_match': 0.6370706416072586,
 'eval_runtime': 104.2207,
 'eval_samples_per_second': 14.805,
 'eval_steps_per_second': 0.47,
 'epoch': 13.0}

In [19]:
def predict(ds: Dataset) -> List[str]:
    preds = trainer.predict(
        ds,
        max_length=MAX_TARGET_LENGTH,
    )

    preds = tokenizer.batch_decode(
        preds.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    return [pred.strip() for pred in preds]


In [20]:
preds = predict(dataset["test"])

preds[:5], dataset["test"]["vega_zero"][:5]


The following columns in the test set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: query, vega_zero, chart, db_id, SQL, hardness, table, question. If query, vega_zero, chart, db_id, SQL, hardness, table, question are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1543
  Batch size = 32


(['mark bar encoding x openning_year y aggregate count openning_year transform group x sort x desc',
  'mark bar encoding x year y aggregate count year transform sort y desc bin x by weekday',
  'mark bar encoding x sex y aggregate min weight transform group x sort y desc',
  'mark bar encoding x sex y aggregate mean weight transform group x sort y desc',
  'mark bar encoding x date_address_from y aggregate count date_address_from transform bin x by month'],
 ['mark bar encoding x openning_year y aggregate count openning_year transform group x sort x desc',
  'mark bar encoding x year y aggregate count year transform group x sort y desc',
  'mark bar encoding x sex y aggregate min weight transform group x sort y desc',
  'mark bar encoding x sex y aggregate mean weight transform group x sort y desc',
  'mark bar encoding x date_address_from y aggregate count date_address_from transform sort monthly_rental desc bin x by year'])

In [21]:
exact_match.compute(
    predictions=preds,
    references=dataset["test"]["vega_zero"],
)


{'exact_match': 0.6331821127673364}

In [22]:
preds_df = dataset["test"].to_pandas()
preds_df = preds_df.drop(columns=["input_ids", "attention_mask", "labels"])
preds_df["pred"] = preds
preds_df["exact_matched"] = preds_df["pred"] == preds_df["vega_zero"]

preds_df.to_csv(RESULT_OUTPUT_DIR.joinpath("prediction.csv"))

preds_df


,db_id,chart,hardness,query,question,vega_zero,SQL,table,pred,exact_matched
0,cinema,Bar,Medium,"Visualize BAR SELECT Openning_year , COUNT(Ope...",give me a bar chart showing the number of cine...,mark bar encoding x openning_year y aggregate ...,"SELECT Openning_year , COUNT(Openning_year) FR...",cinema,mark bar encoding x openning_year y aggregate ...,True
1,wta_1,Bar,Medium,"Visualize BAR SELECT year , count(*) FROM matc...",find the number of matches happened in each ye...,mark bar encoding x year y aggregate count yea...,"SELECT year , count(*) FROM matches GROUP BY Y...",matches,mark bar encoding x year y aggregate count yea...,False
2,candidate_poll,Bar,Easy,"Visualize BAR SELECT Sex , min(weight) FROM pe...",what is the minimum weights for people of each...,mark bar encoding x sex y aggregate min weight...,"SELECT Sex , min(weight) FROM people GROUP BY ...",people,mark bar encoding x sex y aggregate min weight...,True
3,candidate_poll,Bar,Medium,"Visualize BAR SELECT Sex , AVG(Weight) FROM pe...",show me the average of weight by sex in a hist...,mark bar encoding x sex y aggregate mean weigh...,"SELECT Sex , AVG(Weight) FROM people GROUP BY ...",people,mark bar encoding x sex y aggregate mean weigh...,True
4,behavior_monitoring,Bar,Medium,"Visualize BAR SELECT date_address_from , COUNT...",visualize a bar chart about the distribution o...,mark bar encoding x date_address_from y aggreg...,"SELECT date_address_from , COUNT(date_address_...",student_addresses,mark bar encoding x date_address_from y aggreg...,False
...,...,...,...,...,...,...,...,...,...,...
1538,local_govt_in_alabama,Pie,Easy,"Visualize PIE SELECT Event_Details , COUNT(Eve...",group and count details for the events using a...,mark arc encoding x event_details y aggregate ...,"SELECT Event_Details , COUNT(Event_Details) FR...",events,mark arc encoding x event_details y aggregate ...,True
1539,riding_club,Bar,Medium,"Visualize BAR SELECT Occupation , COUNT(Occupa...",bar chart x axis occupation y axis how many oc...,mark bar encoding x occupation y aggregate cou...,"SELECT Occupation , COUNT(Occupation) FROM pla...",player,mark bar encoding x occupation y aggregate cou...,True
1540,store_product,Pie,Easy,"Visualize PIE SELECT Type , count(*) FROM stor...","for each type of store , how many of them are ...",mark arc encoding x type y aggregate count typ...,"SELECT Type , count(*) FROM store GROUP BY TYPE",store,mark arc encoding x type y aggregate count typ...,True
1541,candidate_poll,Bar,Easy,"Visualize BAR SELECT Name , Weight FROM people...",return a bar chart about the distribution of n...,mark bar encoding x name y aggregate none weig...,"SELECT Name , Weight FROM people ORDER BY Name...",people,mark bar encoding x name y aggregate none weig...,True


In [23]:
pd.concat(
    [
        preds_df[preds_df["hardness"] == hardness]["exact_matched"]
        .value_counts()
        .rename(hardness)
        for hardness in ("Easy", "Medium", "Hard", "Extra Hard")
    ],
    axis=1,
)


,Easy,Medium,Hard,Extra Hard
True,452,492,31,2
False,136,277,51,102


In [24]:
pd.concat(
    [
        preds_df[preds_df["chart"] == chart]["exact_matched"]
        .value_counts()
        .rename(chart)
        for chart in preds_df["chart"].unique()
    ],
    axis=1,
)


,Bar,Line,Scatter,Stacked Bar,Grouping Line,Pie,Grouping Scatter
True,776,45,49,12,3,72,20
False,270,83,85,67,24,17,20


## Push Model


In [25]:
if push_model_to_huggingface_hub:
    # huggingface_hub.notebook_login()

    trainer.push_to_hub()


Saving model checkpoint to ../data/models/vxnli-v0
Configuration saved in ../data/models/vxnli-v0/config.json
Model weights saved in ../data/models/vxnli-v0/pytorch_model.bin
tokenizer config file saved in ../data/models/vxnli-v0/tokenizer_config.json
Special tokens file saved in ../data/models/vxnli-v0/special_tokens_map.json
remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/kwkty/vxnli-v0
   a7639c8..e34bb04  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/kwkty/vxnli-v0
   e34bb04..d1f2c27  main -> main

